In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import copy
sys.path.append('/users/mtaranov/LongRange3D/')
from models_motif_exp import LongRangeDNN_FC
from utils import normalize_features_sampl_by_f, concat_motifs,get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 
#from sklearn.preprocessing import MinMaxScaler, StandardScaler
#from dragonn.models import Model, SequenceDNN
#from keras.models import Sequential
#from keras.callbacks import Callback, EarlyStopping
#from keras.layers.core import (
#    Activation, Dense, Dropout, Flatten,
#    Permute, Reshape, TimeDistributedDense
#)
#from keras.layers.convolutional import Convolution2D, MaxPooling2D
#from keras.layers.recurrent import GRU
#from keras.regularizers import l1

#from deeplift import keras_conversion as kc
#from deeplift.blobs import MxtsMode

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 2500 MB, cuDNN 5005)


In [2]:
contacts='PE'
path='/users/mtaranov/datasets_3d/dist_matched_'+contacts+'/'

X_train = np.load(path+'motifs/train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')
X_test = np.load(path+'motifs/test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')
X_valid = np.load(path+'motifs/valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')

y_train = get_labels(path+'y_train_thres_10.npy')
y_test = get_labels(path+'y_test_thres_10.npy')
y_valid = get_labels(path+'y_valid_thres_10.npy')


In [3]:
X_train.shape, X_test.shape, X_valid.shape

((4824, 1920), (1812, 1920), (2872, 1920))

In [4]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features_sampl_by_f(X_train, X_valid, X_test)

#X_train_normalized_subsampled, y_train_subsampled = subsample_data(X_train_normalized, y_train)
#X_valid_normalized_subsampled, y_valid_subsampled = subsample_data(X_valid_normalized, y_valid)
#X_test_normalized_subsampled, y_test_subsampled = subsample_data(X_test_normalized, y_test)

#X_train_scaled_subsampled, y_train_subsampled = subsample_data(X_train_scaled, y_train)
#X_valid_scaled_subsampled, y_valid_subsampled = subsample_data(X_valid_scaled, y_valid)
#X_test_scaled_subsampled, y_test_subsampled = subsample_data(X_test_scaled, y_test)


In [5]:
from models_motif_exp import LongRangeDNN_FC_ResNet
dnn_normalized = LongRangeDNN_FC_ResNet(num_features=1920, use_deep_CNN=False)
validation_data = (X_valid_normalized, y_valid)
dnn_normalized.train(X_train_normalized, y_train, validation_data)

Training model...
Train on 4824 samples, validate on 2872 samples
Epoch 1/100
4800/4824 [============================>.] - ETA: 0s - loss: 1.2136Epoch 0: validation loss: 0.545
Balanced Accuracy: 75.87%	 auROC: 0.837	 auPRC: 0.830	 auPRG: 0.708
Recall at 5%|10%|20% FDR: 0.2%|47.6%|70.0%	 Num Positives: 1436	 Num Negatives: 1436	 

4824/4824 [==============================] - 8s - loss: 1.2117 - val_loss: 0.5446
Epoch 2/100
4800/4824 [============================>.] - ETA: 0s - loss: 0.4878Epoch 1: validation loss: 0.473
Balanced Accuracy: 80.54%	 auROC: 0.885	 auPRC: 0.871	 auPRG: 0.815
Recall at 5%|10%|20% FDR: 0.3%|65.9%|82.1%	 Num Positives: 1436	 Num Negatives: 1436	 

4824/4824 [==============================] - 7s - loss: 0.4872 - val_loss: 0.4732
Epoch 3/100
4800/4824 [============================>.] - ETA: 0s - loss: 0.2711Epoch 2: validation loss: 0.422
Balanced Accuracy: 82.76%	 auROC: 0.907	 auPRC: 0.899	 auPRG: 0.864
Recall at 5%|10%|20% FDR: 67.0%|75.1%|85.5%	 Num Positive

In [6]:
print(dnn_normalized.test(X_test_normalized, y_test))
print(dnn_normalized.test(X_train_normalized, y_train))
print(dnn_normalized.test(X_valid_normalized, y_valid))

Balanced Accuracy: 84.71%	 auROC: 0.918	 auPRC: 0.928	 auPRG: 0.882
Recall at 5%|10%|20% FDR: 69.8%|81.0%|87.2%	 Num Positives: 906	 Num Negatives: 906	 
Balanced Accuracy: 100.00%	 auROC: 1.000	 auPRC: 1.000	 auPRG: 1.000
Recall at 5%|10%|20% FDR: 100.0%|100.0%|100.0%	 Num Positives: 2412	 Num Negatives: 2412	 
Balanced Accuracy: 85.38%	 auROC: 0.930	 auPRC: 0.931	 auPRG: 0.904
Recall at 5%|10%|20% FDR: 75.3%|80.9%|88.4%	 Num Positives: 1436	 Num Negatives: 1436	 
